In [11]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)
t_c = [[0.5,  14.0], [15.0,11.0], [28.0, 11.0],  [8.0,  3.0], [-4.0,  6.0], [13.0, 21.0]]
t_u = [[35.7,33.1], [55.9, 58.2], [81.9, 56.3], [48.9, 33.9], [21.8, 48.4], [60.4, 68.4]]
t_c = torch.tensor(t_c) # <1>
t_u = torch.tensor(t_u)# <1>


In [12]:
t_c.shape

torch.Size([6, 2])

In [13]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

t_un_train , t_un_val

(tensor([[4.8900, 3.3900],
         [6.0400, 6.8400],
         [8.1900, 5.6300],
         [2.1800, 4.8400],
         [3.5700, 3.3100]]),
 tensor([[5.5900, 5.8200]]))

In [14]:
import torch.nn as nn
linear_model = nn.Linear(2, 1) # 입력 피처2 , 출력 피처1 을 가진 nn.Linear 인스턴스 생성
linear_model(t_un_val)
linear_model.weight , linear_model.bias

(Parameter containing:
 tensor([[ 0.0147, -0.3985]], requires_grad=True),
 Parameter containing:
 tensor([0.7005], requires_grad=True))

In [31]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # 10*1
t_u = torch.tensor(t_u).unsqueeze(1) # 10*1

linear_model = nn.Linear(2, 2) # 입력피처1 , 출력피처1
optimizer = optim.SGD(linear_model.parameters(),lr=1e-2)

list(linear_model.parameters()) # init생성자에 정의된 서브 모듈까지 재귀적으로 호출하며 만나는 모든 파마리터 리스트를 담은 리스트

[Parameter containing:
 tensor([[-0.0896,  0.4661],
         [ 0.2957,  0.0953]], requires_grad=True),
 Parameter containing:
 tensor([0.3779, 0.2452], requires_grad=True)]

In [32]:
linear_model

Linear(in_features=2, out_features=2, bias=True)

In [34]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [35]:
training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(), # <1>
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

linear_model.weight , linear_model.bias

Epoch 1, Training loss 141.8551, Validation loss 113.5386
Epoch 1000, Training loss 15.7817, Validation loss 10.4365
Epoch 2000, Training loss 13.6056, Validation loss 9.0250
Epoch 3000, Training loss 13.0500, Validation loss 8.4199


(Parameter containing:
 tensor([[ 5.2724, -0.3279],
         [ 0.0117,  2.8768]], requires_grad=True),
 Parameter containing:
 tensor([-15.3951,  -2.8375], requires_grad=True))

In [10]:
import torch.nn as nn
seq_model2 = nn.Sequential(
    nn.Linear(1,13),
    nn.Tanh(),
    nn.Linear(13,1)
)
seq_model2


Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [10]:
for name , params in seq_model2.named_parameters():
    print(name , params , params.shape) # 처음에는 임의의 값 배정

0.weight Parameter containing:
tensor([[ 0.4678],
        [-0.1260],
        [-0.2810],
        [ 0.7049],
        [ 0.3439],
        [ 0.4722],
        [-0.7408],
        [ 0.5509],
        [-0.1674],
        [ 0.7639],
        [ 0.5701],
        [ 0.6725],
        [ 0.1320]], requires_grad=True) torch.Size([13, 1])
0.bias Parameter containing:
tensor([-0.0222,  0.2186, -0.3544, -0.0907,  0.8263, -0.5409,  0.4232,
         0.2743, -0.0480,  0.2576,  0.7402,  0.7180, -0.9278],
       requires_grad=True) torch.Size([13])
2.weight Parameter containing:
tensor([[-0.1877,  0.0069,  0.2000, -0.0582, -0.1776,  0.1916, -0.0504,
          0.0743, -0.1032, -0.1246,  0.0963,  0.2261, -0.0381]],
       requires_grad=True) torch.Size([1, 13])
2.bias Parameter containing:
tensor([0.0278], requires_grad=True) torch.Size([1])


In [40]:
from collections import OrderedDict
seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(2, 10)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(10, 2))
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=3, out_features=10, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=10, out_features=3, bias=True)
)

In [12]:
seq_model.hidden_linear.bias , seq_model.hidden_linear.bias.shape , seq_model.hidden_linear.weight , seq_model.hidden_linear.weight.shape

(Parameter containing:
 tensor([ 0.1201,  0.2730,  0.3547,  0.5108, -0.4209, -0.4567,  0.2949,
         -0.2065, -0.2110, -0.2667], requires_grad=True),
 torch.Size([10]),
 Parameter containing:
 tensor([[ 0.2081, -0.5624,  0.0811],
         [ 0.2154, -0.2787, -0.2539],
         [ 0.2454,  0.5625, -0.5195],
         [ 0.5112, -0.2309, -0.0689],
         [ 0.3663, -0.4153,  0.3244],
         [-0.0218,  0.4839,  0.5064],
         [-0.0578, -0.2637,  0.0118],
         [ 0.1861, -0.3329,  0.4281],
         [ 0.3326, -0.1494, -0.0889],
         [-0.2621,  0.2971,  0.4496]], requires_grad=True),
 torch.Size([10, 3]))

In [13]:
seq_model.output_linear.bias , seq_model.output_linear.bias.shape , seq_model.output_linear.weight , seq_model.output_linear.weight.shape

(Parameter containing:
 tensor([0.1020], requires_grad=True),
 torch.Size([1]),
 Parameter containing:
 tensor([[-0.2180,  0.1208,  0.1115,  0.1197,  0.1042, -0.0606,  0.1916,
          -0.0764, -0.1128, -0.3003]], requires_grad=True),
 torch.Size([1, 10]))

In [7]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer, 
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)
    
print('output', seq_model(t_un_val))
print('answer', t_c_val)
# print('hidden', seq_model.hidden_linear.weight.grad)

NameError: name 'seq_model' is not defined